# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA ) em 2 Etapas

In [0]:
# Importação dos pacotes para Analise

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Importando pacotes para treino e validação do modelo

from sklearn.feature_extraction.text import TfidfVectorizer
# um classificador linear que utiliza o Gradiente Descendente Estocástico como método de treino. 
# Por padrão, utiliza o estimador SVM.
from sklearn.linear_model import SGDClassifier
# Uma rede neural Perceptron Multicamadas
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split



In [0]:
%matplotlib inline

In [0]:
#Carregar os dados para o google colab
uploaded = files.upload()

Saving Hackathon_Base_Teste.csv to Hackathon_Base_Teste.csv
Saving Hackathon_Base_Treino_comdep.csv to Hackathon_Base_Treino_comdep.csv


In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')


### Análise dos dados

In [0]:
# Exemplos dos dados treino

df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA


In [0]:
# Exemplos dos dados treino teste

df2.head(2)

,0,DESCRIÇÃO PARCEIRO
0,1,SAL ROSA HIMALAIA C/ ALHO 500G
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952


In [0]:
# Verificação de nulos

df.isna().sum()

DESCRIÇÃO PARCEIRO    0
SUB-CATEGORIA         0
CATEGORIA             0
DEPARTAMENTO          0
dtype: int64

In [0]:
# Tamanho do dataset de treino (Vamos usar para treino e teste afim de validar os dados)

df.shape

(22009, 4)

In [0]:
# Quantidade das SUB-CATEGORIA, das CATEGORIA e das DEPARTAMENTO

classifica = df[['DEPARTAMENTO','CATEGORIA', 'SUB-CATEGORIA']]
classifica.nunique()

DEPARTAMENTO       53
CATEGORIA         332
SUB-CATEGORIA    1968
dtype: int64

In [0]:
# Tamanho das descrições

np.sort(df['DESCRIÇÃO PARCEIRO'].str.len().unique())

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 63, 64, 65, 67, 69, 70, 71, 74])

In [0]:
# Tamanho médio das descrições

df['DESCRIÇÃO PARCEIRO'].str.len().unique().mean()

37.292307692307695

### Etapa de Preparação do Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)

In [0]:
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C/1


In [0]:
df.query('DEPARTAMENTO == "CUIDADOS COM OS OLHOS"')

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA
3141,MASCARA CILIOS VULT SUPER LASHES 12G,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASCARA CILIOS VULT SUPER LASHES
5866,MASCARA CILIOS MAYB COLOSSAL S FILM NU,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASCARA CILIOS MAYB COLOSSAL S FILM NU
8888,MASC CILIOS BE AURORA LAMA NEGRA ALONGA,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASC CILIOS BE AURORA LAMA NEGRA ALONGA
10133,MASCARA CILIOS VULT PERFECT LASHES PT 7G,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASCARA CILIOS VULT PERFECT LASHES PT
10285,MASCARA CILIOS PAYOT BOC ROS #MEUVOLM 6G,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASCARA CILIOS PAYOT BOC ROS #MEUVOLM
11470,MASC CILIOS ARCANCIL PARIS 001 BLACK 1UN,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASC CILIOS ARCANCIL PARIS
11818,MASC CILIOS ARCANCIL PTO 049 1UN,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASC CILIOS ARCANCIL PTO
12453,MASCARA CILIOS MAYB TOTAL TEMP PROVA PT,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASCARA CILIOS MAYB TOTAL TEMP PROVA PT
19970,MASC CILIOS VULT ALONGADORA BL,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASC CILIOS VULT ALONGADORA BL
21400,MASC CILIOS MAYBELLINE FALSIES PTO POSTISO WS,MÁSCARA DE CÍLIOS,MAQUIAGEM,CUIDADOS COM OS OLHOS,MASC CILIOS MAYBELLINE FALSIES PTO POSTISO WS


### Etapa 1 - Classificando CATEGORIA

In [0]:
#Criando a nova para classificar CATEGORIA

base = df

In [0]:
# Selecionando apenas o item a ser classificado e o target do DF principal

base_data = base['DESC_AJUSTADA']
base_target = base['CATEGORIA']

In [0]:
# Transformando a descrição em vetor
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
vetor = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x = vetor.fit_transform(base_data)

In [0]:
# Dividindo dataset em treino e teste

x_train, x_test, y_train, y_test = train_test_split(vetor_x, base_target, test_size= 0.25, random_state=27)

In [0]:
# Aqui nós treinamos o classificador

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(500, ), random_state=1, verbose=True)

In [0]:
clf.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [0]:
# Avaliando a performance com predição

predicted = clf.predict(x_test)

print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))

# Matriz de confusão
print('\n### Matriz de Confusão x###\n')
print(metrics.confusion_matrix(y_test, predicted))



###Indicadores Classificação###

                                           precision    recall  f1-score   support

                               ABSORVENTE       1.00      0.94      0.97        18
                      ACESSÓRIOS DE BANHO       0.71      0.86      0.77        14
                 ACESSÓRIOS DE LAVANDERIA       0.93      0.93      0.93        14
                 ACESSÓRIOS DE MANUTENÇÃO       0.50      0.50      0.50         2
                        ACESSÓRIOS ELETRO       0.92      0.58      0.71        19
                      ACESSÓRIOS MÃO E PÉ       0.92      0.69      0.79        16
                   ACESSÓRIOS PARA CABELO       0.94      0.94      0.94        32
                           ACESSÓRIOS PET       0.93      0.72      0.81        18
                                 ADOÇANTE       1.00      1.00      1.00         5
                       AGUARDENTE/CACHAÇA       0.77      1.00      0.87        10
                                  ALGODÃO       0.50

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# Predição em novos dados com o DF  de teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
base_novos = df2['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n = vetor.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(vetor_n)


In [0]:
# Adicionando a coluna para o novo DF
df_cat = df2[['DESCRIÇÃO PARCEIRO','DESC_AJUSTADA']]
df_cat['CATEGORIA'] = predicted_novos

In [0]:
# Novos Dados de teste incluindo o departamento
df_cat.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C/ ALHO,SAL
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C/,ENFEITES DE NATAL


### Etapa 2 - Classificando SUB-CATEGORIA

In [0]:
# Criando a nova base para classificar CATEGORIA

base2 = df

# Selecionando apenas o item a ser classificado e o target do DF principal

base_data2 = base2['CATEGORIA']+' '+base2['DESCRIÇÃO PARCEIRO']
base_target2 = base2['SUB-CATEGORIA']

In [0]:
# Transformando a descrição em vetor

stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','p/','la','&','.',',','-']
vetor2 = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x2 = vetor2.fit_transform(base_data2)
vetor_x2

<22009x17111 sparse matrix of type '<class 'numpy.float64'>'
	with 148833 stored elements in Compressed Sparse Row format>

In [0]:
# Dividindo dataset em treino e teste

x_train2, x_test2, y_train2, y_test2 = train_test_split(vetor_x2, base_target2, test_size=0.25, random_state=27)


In [0]:
# Aqui nós treinamos o classificador

clf2 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(500, ), random_state=1, verbose=True)
clf2.fit(x_train2, y_train2)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [0]:
# Avaliando a performance com predição

predicted2 = clf2.predict(x_test2)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test2, predicted2))

# Matriz de confusão
print('\n### Matriz de Confusão Categoria###\n')
print(metrics.confusion_matrix(y_test2, predicted2))


###Indicadores Classificação Categoria###

                                          precision    recall  f1-score   support

                                  2 EM 1       0.00      0.00      0.00         1
                              20 VOLUMES       0.00      0.00      0.00         1
                              40 VOLUMES       0.00      0.00      0.00         2
                                      4K       1.00      0.86      0.92         7
                                 ABACATE       0.00      0.00      0.00         2
                                 ABACAXI       1.00      0.86      0.92         7
                               ABOBRINHA       1.00      0.50      0.67         2
                      ABOBRINHA ORGÂNICA       0.50      1.00      0.67         1
                                ABRIDOR        0.67      1.00      0.80         2
                                 ABÓBORA       0.86      1.00      0.92         6
                        ABÓBORA ORGÂNICA       0.00  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# Predição em novos dados utilizando o df_cat

base_novos2 = df_cat['CATEGORIA']+' '+df_cat['DESCRIÇÃO PARCEIRO']

# Tranformando descrição em vetor
vetor_n2 = vetor2.transform(base_novos2)

#Previsão dos novos dados
predicted_novos2 = clf2.predict(vetor_n2)


In [0]:
# Adicionando a coluna no novo DF
df_cat_sub = df_cat
df_cat_sub['SUB-CATEGORIA'] = predicted_novos2

In [0]:
# Novos Dados de teste incluindo o departamento
df_cat_sub.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,CATEGORIA,SUB-CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C/ ALHO,SAL,SAL DO HIMALAIA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C/,ENFEITES DE NATAL,ENFEITES DE ÁRVORE DE NATAL


### Referências

https://medium.com/luisfredgs/classificando-textos-com-machine-learning-e054ca7bf4e0

https://www.alura.com.br/artigos/classificando-textos-com-python

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html